# HW2

In [58]:
# import libiaries
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import pandas as pd
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /Users/zazhu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read Yelp review data

In [28]:
review = pd.read_json('yelp_academic_dataset_review.json',lines=True)

In [48]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


## Functions

In [148]:
def preprocess(df):
    '''tokenization, normalization (e.g. convert to lowercase, remove non-alphanumeric chars, numbers, etc.）'''
    text_norm = []
    text_norm_string = []
    for i in range(0,len(df)): 
        row = df.iloc[[i]]
        strings = row.to_string(header = False)       
        # tokenize 
        token = word_tokenize(strings)
        # only keep alphabet and get lower case
        alpha = [word.lower() for word in token if word.isalpha()]
        # nomolized taxt list 
        text_norm.append(alpha)
        text_norm_string.append(" ".join(alpha))
    # Return to text file
    with open('yelp_text_norm.txt', 'w') as f:
        for l in text_norm:
            f.write("%s\n" % l)
    
    return(text_norm)

# find top k closest neighbors of target words under given models 
def find_neighbor(model, words, k):
    dic = {}
    for word in words:
        dic[word] = [tuple(x) for x in pd.DataFrame(model.wv.most_similar(word))[:k].round(2).values]
    
    model_df = pd.DataFrame(dic)
    
    return model_df  

## Preprocess

In [88]:
# data is too large, choose only 100k records of data
nomalized_text = preprocess(review.head(100000))

In [145]:
nomalized_text[0][:10]

['as',
 'someone',
 'who',
 'has',
 'worked',
 'with',
 'many',
 'museums',
 'i',
 'was']

In [125]:
#word2vec models with varying parameters
model_cbow1 = gensim.models.Word2Vec(nomalized_text, window=5, workers=5, sg=0)
model_cbow2 = gensim.models.Word2Vec(nomalized_text, window=50, workers=5, sg=0)
model_sg = gensim.models.Word2Vec(nomalized_text, window=50, workers=5, sg=1)

In [149]:
# find top k=5 closest neighbors of 10 target words under given models 
words_10 = ["expensive", "summer", "hotpot", "husband","hotel", "salad", "chicago", "coffee", "gay","wonderful"]

model1_similar = find_neighbor(model_cbow1, words_10, k=5)
model2_similar = find_neighbor(model_cbow2, words_10, k=5)
model3_similar = find_neighbor(model_sg, words_10, k=5)

In [150]:
model1_similar

,expensive,summer,hotpot,husband,hotel,salad,chicago,coffee,gay,wonderful
0,"(pricey, 0.78)","(winter, 0.87)","(szechuan, 0.74)","(wife, 0.95)","(casino, 0.77)","(hummus, 0.67)","(nyc, 0.81)","(brew, 0.72)","(biker, 0.71)","(fantastic, 0.89)"
1,"(overpriced, 0.73)","(season, 0.7)","(mee, 0.73)","(boyfriend, 0.95)","(flamingo, 0.74)","(falafel, 0.65)","(hawaii, 0.76)","(tea, 0.71)","(hipsters, 0.7)","(fabulous, 0.87)"
2,"(pricy, 0.71)","(evenings, 0.67)","(manchurian, 0.73)","(bf, 0.92)","(trump, 0.72)","(sandwich, 0.65)","(seattle, 0.73)","(latte, 0.71)","(folk, 0.69)","(terrific, 0.82)"
3,"(steep, 0.7)","(summers, 0.65)","(dumpling, 0.73)","(fiancé, 0.9)","(motel, 0.7)","(kale, 0.64)","(boston, 0.73)","(cappuccino, 0.7)","(bearded, 0.68)","(phenomenal, 0.82)"
4,"(cheap, 0.65)","(weather, 0.65)","(dolsot, 0.72)","(girlfriend, 0.9)","(resort, 0.7)","(spinach, 0.64)","(cali, 0.72)","(espresso, 0.67)","(frat, 0.67)","(superb, 0.81)"


In [151]:
model2_similar

,expensive,summer,hotpot,husband,hotel,salad,chicago,coffee,gay,wonderful
0,"(pricey, 0.7)","(winter, 0.82)","(bim, 0.61)","(boyfriend, 0.8)","(resort, 0.79)","(dressing, 0.82)","(nyc, 0.7)","(latte, 0.77)","(females, 0.63)","(fabulous, 0.83)"
1,"(overpriced, 0.65)","(weather, 0.74)","(sichuan, 0.6)","(hubby, 0.78)","(trump, 0.78)","(croutons, 0.76)","(ohio, 0.65)","(cappuccino, 0.77)","(carey, 0.61)","(fantastic, 0.83)"
2,"(steep, 0.61)","(sun, 0.66)","(congee, 0.59)","(fiancé, 0.78)","(hotels, 0.78)","(vinaigrette, 0.74)","(brooklyn, 0.65)","(espresso, 0.77)","(booty, 0.6)","(phenomenal, 0.79)"
3,"(outrageous, 0.57)","(evenings, 0.66)","(gai, 0.59)","(fiance, 0.74)","(hilton, 0.76)","(romaine, 0.74)","(coast, 0.65)","(mocha, 0.77)","(costumes, 0.6)","(incredible, 0.78)"
4,"(paying, 0.57)","(season, 0.65)","(bi, 0.59)","(son, 0.74)","(staying, 0.74)","(kale, 0.72)","(pittsburgh, 0.64)","(coffees, 0.74)","(songs, 0.6)","(outstanding, 0.73)"


In [152]:
model3_similar

,expensive,summer,hotpot,husband,hotel,salad,chicago,coffee,gay,wonderful
0,"(pricey, 0.84)","(winter, 0.81)","(steamed, 0.73)","(wife, 0.87)","(hotels, 0.91)","(dressing, 0.91)","(giordano, 0.76)","(cappuccino, 0.83)","(dance, 0.71)","(fantastic, 0.9)"
1,"(considering, 0.82)","(weather, 0.77)","(dishes, 0.72)","(hubby, 0.84)","(stay, 0.9)","(vinaigrette, 0.84)","(coast, 0.72)","(latte, 0.83)","(shirtless, 0.71)","(fabulous, 0.89)"
2,"(price, 0.81)","(sun, 0.71)","(dumplings, 0.72)","(boyfriend, 0.84)","(staying, 0.89)","(croutons, 0.84)","(giordanos, 0.72)","(lattes, 0.83)","(dancers, 0.7)","(amazing, 0.86)"
3,"(than, 0.81)","(outdoor, 0.7)","(noodle, 0.71)","(my, 0.8)","(rooms, 0.87)","(lettuce, 0.84)","(pizzerias, 0.71)","(coffees, 0.82)","(tunes, 0.7)","(phenomenal, 0.85)"
4,"(overpriced, 0.81)","(during, 0.67)","(tofu, 0.7)","(had, 0.8)","(resort, 0.87)","(chopped, 0.84)","(pittsburgh, 0.71)","(espresso, 0.81)","(gaga, 0.7)","(incredible, 0.85)"
